In [1]:
import pandas as pd
import numpy as np
import datetime
import hashlib
import glob

In [2]:
samplerows=None
# The lower frmindex, the better

In [3]:
data_1_Jan=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/Segment_Movement_analysis/Data_From_Sp/dfrfm.csv",
                      nrows=samplerows,dtype=str)
data_1_Jan['customer_id_hashed'] = data_1_Jan['Customer_id'].apply(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest())

data_2_July=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/Segment_Movement_analysis/Data_From_Sp/df_crm_finalscore_0714data.csv",
                      nrows=samplerows,dtype=str)

In [4]:
data_2_July['frmindex'].unique().tolist()

['D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10']

In [5]:
data_3_Jan_T_C_list=glob.glob("/home/jian/Projects/Big_Lots/Loyal_members/Segment_Movement_analysis/Data_From_Sp/LRIDs/*.csv")

In [6]:
Jan_T_C_df=pd.DataFrame()
for file in data_3_Jan_T_C_list:
    df=pd.read_csv(file,dtype=str)
    df=df[['LRSegmentID','Customer_id']]
    Jan_T_C_df=Jan_T_C_df.append(df)
Jan_T_C_df['customer_id_hashed']=Jan_T_C_df['Customer_id'].apply(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest())

Jan_group=Jan_T_C_df[['customer_id_hashed','LRSegmentID']].rename(columns={"LRSegmentID":"Jan_Group"})

In [7]:
df_Jan=data_1_Jan[['customer_id_hashed','frmindex']].drop_duplicates()
df_July=data_2_July[['customer_id_hashed','frmindex','finaltest']].drop_duplicates(['customer_id_hashed','frmindex'])
df_Jan=df_Jan.rename(columns={"frmindex":"Jan_Label"})
df_July=df_July.rename(columns={"frmindex":"July_Label",'finaltest':"July_Group"})
df_Jan=pd.merge(df_Jan,Jan_group,on="customer_id_hashed",how="left")
print("unique_id count in Jan: "+str(len(df_Jan['customer_id_hashed'].unique())))
print("unique_id count in July: "+str(len(df_July['customer_id_hashed'].unique())))

unique_id count in Jan: 17796102
unique_id count in July: 20850534


In [85]:
both_df=pd.merge(df_Jan,df_July,on="customer_id_hashed",how="inner")

both_df['Jan_Label']=both_df['Jan_Label'].astype(float).astype(int).astype(str)
both_df['Jan_Label']=both_df['Jan_Label'].apply(lambda x: x.zfill(2))
both_df['Jan_Label']=both_df['Jan_Label'].apply(lambda x: "Jan_"+x)

both_df['July_Label']=both_df['July_Label'].apply(lambda x: x.replace("D","July_"))
print("unique_id count in both: "+str(len(both_df['customer_id_hashed'].unique())))

unique_id count in both: 17791913


In [87]:
both_df['Jan_Group']=both_df['Jan_Group'].fillna("OOOOOO")
both_df['Jan_Group']=both_df['Jan_Group'].apply(lambda x: x[0])
both_df['Jan_Group']=both_df['Jan_Group'].replace("O",np.nan)

In [88]:
both_df['Jan_Group'].unique()

array([nan, 'C', 'T'], dtype=object)

# 1. 10*10

In [9]:
count_by_label=both_df.groupby(['Jan_Label','July_Label'])['customer_id_hashed'].count().to_frame().reset_index()
count_by_label['customer_id_hashed']=count_by_label['customer_id_hashed'].astype(int)
output_1=count_by_label.pivot(index="Jan_Label",columns="July_Label",values="customer_id_hashed")
output_1=output_1.fillna(0)
output_1=output_1.reset_index()

# 2. 30 * 10

In [10]:
df_id_zip=data_2_July[['customer_id_hashed','customer_zip_code']].drop_duplicates()
df_id_zip=df_id_zip.drop_duplicates(['customer_id_hashed'])
both_df=pd.merge(both_df,df_id_zip,on="customer_id_hashed",how="left")

In [11]:
zip_PST=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/BL_Zips in new TA (TA level)_JL_20180330.xlsx",dtype=str)
zip_PST=zip_PST[['zip_cd','revenue_flag']].drop_duplicates()
zip_PST['zip_cd']=zip_PST['zip_cd'].apply(lambda x: x.zfill(5))

In [12]:
both_df=both_df.rename(columns={"customer_zip_code":"zip_cd"})
both_df=pd.merge(both_df,zip_PST,on="zip_cd",how="left")
both_df['revenue_flag']=both_df['revenue_flag'].fillna("X")
# Customers living outside of the TA is excluded, as about 8%

In [13]:
output_2=both_df[['Jan_Label','revenue_flag','July_Label','customer_id_hashed']]
output_2=output_2[output_2['revenue_flag']!="X"]
print("both existing and in TA: "+str(len(output_2['customer_id_hashed'].unique())))
output_2['Jan_30_Seg']=output_2['Jan_Label']+"_"+output_2['revenue_flag']
output_2=output_2.groupby(['Jan_30_Seg','July_Label'])['customer_id_hashed'].count().to_frame().reset_index()
output_2=output_2.pivot(index="Jan_30_Seg",columns="July_Label",values="customer_id_hashed")
output_2=output_2.fillna(0)
output_2=output_2.reset_index()

both existing and in TA: 16278669


# 3, 4, 5 Sales/Trans/Sales_per_id_avg 10*10

In [15]:
rewards_sales_from_SP=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/From_Sp/combinedtransactions_0811.csv",
                                  dtype=str,usecols=['customer_id_hashed','transaction_date','transaction_id','total_transaction_amt'])
rewards_sales_from_SP['transaction_date']=rewards_sales_from_SP['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
rewards_sales_from_SP=rewards_sales_from_SP[(rewards_sales_from_SP['transaction_date']>=datetime.date(2018,2,1)) & (rewards_sales_from_SP['transaction_date']<=datetime.date(2018,7,14))]
rewards_sales_from_SP=rewards_sales_from_SP.drop_duplicates()
rewards_sales_from_SP=rewards_sales_from_SP[rewards_sales_from_SP['customer_id_hashed'].isin(both_df['customer_id_hashed'].tolist())]
rewards_sales_from_SP['total_transaction_amt']=rewards_sales_from_SP['total_transaction_amt'].astype(float)

# [Feb 1, July 14]
output_345=pd.merge(rewards_sales_from_SP,both_df[['customer_id_hashed','Jan_Label','July_Label']],on="customer_id_hashed",how="left")


In [34]:
output_3=output_345.groupby(['Jan_Label','July_Label'])['total_transaction_amt'].count().to_frame().reset_index()
output_3=output_3.pivot(index="Jan_Label",columns="July_Label",values="total_transaction_amt")
output_3=output_3.fillna(0)
output_3=output_3.reset_index()

July_Label,Jan_Label,July_01,July_02,July_03,July_04,July_05,July_06
0,Jan_01,7528188.0,110378.0,1018208.0,2.0,152035.0,0.0
1,Jan_02,1015038.0,492774.0,892674.0,188736.0,182462.0,12162.0
2,Jan_03,2896304.0,246374.0,1112605.0,32430.0,187129.0,0.0
3,Jan_04,218929.0,412851.0,311560.0,309717.0,100326.0,78698.0
4,Jan_05,901335.0,337650.0,791390.0,118064.0,171256.0,2091.0
5,Jan_06,98881.0,330933.0,172149.0,302213.0,72930.0,78315.0
6,Jan_07,320882.0,259987.0,462383.0,151563.0,130649.0,9044.0
7,Jan_08,43077.0,200609.0,84947.0,205814.0,42680.0,80805.0
8,Jan_09,72162.0,175826.0,153215.0,145029.0,64696.0,27815.0
9,Jan_10,25336.0,147784.0,48340.0,153233.0,22517.0,75877.0


In [37]:
output_4=output_345.groupby(['Jan_Label','July_Label'])['total_transaction_amt'].sum().to_frame().reset_index()
output_4=output_4.pivot(index="Jan_Label",columns="July_Label",values="total_transaction_amt")
output_4=output_4.fillna(0)
output_4=output_4.reset_index()

July_Label,Jan_Label,July_01,July_02,July_03,July_04,July_05,July_06
0,Jan_01,2.391731e+08,8.140600e+05,3.809742e+07,3.200000e+00,5.672621e+06,0.00
1,Jan_02,3.567631e+07,1.451912e+07,3.886208e+07,3.869927e+06,7.813778e+06,243840.06
2,Jan_03,1.047471e+08,5.274563e+06,4.441070e+07,4.541417e+05,7.349514e+06,0.00
3,Jan_04,8.778669e+06,1.498366e+07,1.761445e+07,9.205796e+06,6.470128e+06,1538712.24
4,Jan_05,3.739923e+07,1.183640e+07,3.665853e+07,2.578766e+06,7.688876e+06,39167.53
5,Jan_06,4.368999e+06,1.227773e+07,1.118177e+07,8.849586e+06,4.899676e+06,1595931.79
6,Jan_07,1.498880e+07,1.030408e+07,2.466285e+07,3.853408e+06,6.772219e+06,187355.31
7,Jan_08,2.022357e+06,8.401258e+06,6.549180e+06,7.449692e+06,4.272454e+06,1690156.35
8,Jan_09,3.733680e+06,8.415876e+06,1.043330e+07,5.425766e+06,4.435528e+06,575327.88
9,Jan_10,1.190860e+06,6.250257e+06,3.984704e+06,5.746455e+06,2.859220e+06,1623276.03


In [55]:
output_5=output_4.copy()
for i in range(6):
    col_num=i+1
    
    for j in range(len(output_4)):
        if output_1.iloc[j,col_num]!=0:
            output_5.iloc[j,col_num]=output_4.iloc[j,col_num]/output_1.iloc[j,col_num]
        else:
            output_5.iloc[j,col_num]=np.nan

In [60]:
# To be done, T, C, volocity

# 6,7 Test/Controal group movement

In [99]:
both_df_Jan_T=both_df[both_df['Jan_Group']=="T"]
both_df_Jan_C=both_df[both_df['Jan_Group']=="C"]

In [100]:
both_df_Jan_T=both_df_Jan_T.groupby(['Jan_Label','July_Label'])['customer_id_hashed'].count().to_frame().reset_index()
both_df_Jan_T['customer_id_hashed']=both_df_Jan_T['customer_id_hashed'].astype(int)
output_6=both_df_Jan_T.pivot(index="Jan_Label",columns="July_Label",values="customer_id_hashed")
output_6=output_6.fillna(0)
output_6=output_6.reset_index()

In [101]:
both_df_Jan_C=both_df_Jan_C.groupby(['Jan_Label','July_Label'])['customer_id_hashed'].count().to_frame().reset_index()
both_df_Jan_C['customer_id_hashed']=both_df_Jan_C['customer_id_hashed'].astype(int)
output_7=both_df_Jan_C.pivot(index="Jan_Label",columns="July_Label",values="customer_id_hashed")
output_7=output_7.fillna(0)
output_7=output_7.reset_index()

# 8 Movement Vocility

In [113]:
output_8=pd.DataFrame()

for i in range(len(output_1)):
    Jan_Decile=output_1['Jan_Label'][i]
    Jan_Decile=int(Jan_Decile.split("_")[1])
    
    Jan_Decile_Total=output_1[output_1.columns.tolist()[1:]].sum(axis=1)[i]
    
    move_up_ids=0
    move_stay_ids=0
    move_down_ids=0
    
    for j in range(1,output_1.shape[1]):
        July_Decile=int(output_1.columns.tolist()[j].split("_")[1])
        if July_Decile<Jan_Decile:
            move_up_ids=move_up_ids+output_1.iloc[i,j]
        if July_Decile==Jan_Decile:
            move_stay_ids=move_stay_ids+output_1.iloc[i,j]
        if July_Decile>Jan_Decile:
            move_down_ids=move_down_ids+output_1.iloc[i,j]
            
    df=pd.DataFrame({"Jan_Group":output_1['Jan_Label'][i],"Jan_Id_Count":Jan_Decile_Total,"Move_Up_count":move_up_ids,
                    "No_Move_count":move_stay_ids,"Move_Down_count":move_down_ids},index=[i])
    output_8=output_8.append(df)
output_8['Move_Up_Pctg']=output_8['Move_Up_count']/output_8['Jan_Id_Count']
output_8['No_Move_Pctg']=output_8['No_Move_count']/output_8['Jan_Id_Count']
output_8['Move_Dowan_Pctg']=output_8['Move_Down_count']/output_8['Jan_Id_Count']
output_8=output_8[['Jan_Group','Jan_Id_Count','Move_Up_count','No_Move_count','Move_Down_count','Move_Up_Pctg','No_Move_Pctg','Move_Dowan_Pctg']]


In [114]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Loyal_members/Segment_Movement_analysis/output/BL_Rewards members segment movement_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

output_1.to_excel(writer,"Matrix_10_by_10",index=False)
output_2.to_excel(writer,"Matrix_30_by_10",index=False)
output_3.to_excel(writer,"Transaction_10_by_10",index=False)
output_4.to_excel(writer,"Revenue_10_by_10",index=False)
output_5.to_excel(writer,"Avg_Revenue_10_by_10",index=False)
output_6.to_excel(writer,"Jan_T_Movement",index=False)
output_7.to_excel(writer,"Jan_C_Movement",index=False)
output_8.to_excel(writer,"Movement_Pctg",index=False)
writer.save()
